In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import os
import requests
import time
import math
from ultralytics import YOLO
import csv
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
cd /content/drive/MyDrive/Ponds_Wells_Plantations_Detection/Plantations2.0

/content/drive/MyDrive/Pond/Plantations2.0


In [ ]:
pwd = os.getcwd()
pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

# Visualize

In [ ]:
import geemap
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]
easy_scurbland = [
    ee.Filter.gte("size", 60000),
    ee.Filter.gt("red", 0.9)
    ]
easy_plantation = [
    ee.Filter.lt("area", 20000),
    ee.Filter.gt("area", 1000)
]
state = "rajasthan"
district = "sirohi"
block = "pindwara"

#state = "karnataka"
#district = "raichur"
#block = "devadurga"

#state = "rajasthan"
#district = "bhilwara"
#block = "mandalgarh"

#state = "odisha"
#district = "koraput"
#block = "jaypur"
#mandalgarh, jeypur


asset_string = state + "/" + district + "/" + block + "/" + district + "_" + block

original_boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_boundaries")
farm = original_boundaries.filter(ee.Filter.And(*easy_farm))
scrubland = original_boundaries.filter(ee.Filter.And(easy_scurbland))
plantation = original_boundaries.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))


lulc_v4 = ee.Image("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_lulc_v4")
lulc_v3 = ee.Image("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_2023-07-01_2024-06-30_LULCmap_10m")
#lulc_devadurga = ee.Image("projects/ee-corestackdev/assets/apps/mws/karnataka/raichur/devadurga/raichur_devadurga_lulc_v4")
boundaries = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_boundaries_refined")

m = geemap.Map()
m.centerObject(lulc_v4)


url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
m.layout.height = '1000px'
m.add_tile_layer(url, name="Google Map", attribution="Google")
#m.add_basemap(, )
m.addLayerControl()
#m.addLayer(fields_boundary, {}, 'Fields')

palette = [
    '303000', '000000', '000000', '000000', '000000', 'f75cff',
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]

palette_corrected = [
    '000000', '000000', '000000', '000000', '000000', 'f75cff',
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e', '29554E'
]

vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}

vis_params_corrected = {
    'min': 0,
    'max': 13,
    'palette': palette_corrected
}

labels = """0. Background
1. Built-up
2. Water (Kharif)
3. Water (Kharif + Rabi)
4. Water (Kharif + Rabi + Zaid)
5. Croplands
6. Tree/Forest
7. Barren Land
8. Single Kharif (Light blue)
9. Single Non-Kharif
10. Double (Dark-Blue)
11. Triple
12. Shrub_Scrub
13. Plantation"""
labels = [i.split(". ")[-1] for i in labels.split("\n")]


m.addLayer(lulc_v4.select("predicted_label"), vis_params_corrected, 'LULC_v4')
m.addLayer(lulc_v3.select("predicted_label"), vis_params_corrected, 'LULC_v3')
m.addLayer(farm, {}, 'farms_easy_positives')
m.addLayer(plantation, {}, 'plantation_easy_positives')
m.addLayer(scrubland, {}, 'scrubland_easy_positives')
m.addLayer(original_boundaries, {}, "all_boundaries")

#m.addLayer(boundaries_sirohi.filter(ee.Filter.eq('class', "scrubland")), {}, 'Boundaries_sirohi')

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Download using Lat and Lon (XTile and YTile)

In [ ]:
pwd

'/content/drive/MyDrive/Pond'

In [ ]:
# URL for Google map tiles (satellite imagery)
base_url = "https://mt1.google.com/vt/lyrs=s"
zoom_level = 17  # Set the zoom level

# Specify the folder where you want to save the downloaded tiles
output_folder = "C:/Users/Aatif/OneDrive/Desktop"


In [ ]:
# Function to download a specific map tile based on x and y coordinates
def download_map_tile(base_url, output_folder, zoom_level, x_tile, y_tile):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get start time
    start_time = time.time()

    # Construct the URL for the current tile
    tile_url = f"{base_url}&x={x_tile}&y={y_tile}&z={zoom_level}"
    print(tile_url)

    try:
        # Send HTTP GET request to fetch the tile
        response = requests.get(tile_url)

        # Check if request was successful (status code 200)
        if response.status_code == 200:
            # Save the tile image to a file in the output folder
            filename = f"tile_{zoom_level}_{x_tile}_{y_tile}.png"
            filepath = os.path.join(output_folder, filename)
            with open(filepath, "wb") as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download tile ({x_tile}, {y_tile}), HTTP status code: {response.status_code}")

    except Exception as e:
        print(f"Error downloading tile ({x_tile}, {y_tile}): {e}")

    # Get end time
    end_time = time.time()

    # Print the total execution time
    print(f"Total time taken: {end_time - start_time} seconds")



def lat_lon_to_tile(lat, lon, zoom):
    lat = max(min(lat, 85.05112878), -85.05112878)

    # Calculate x tile
    x_tile = int((lon + 180.0) / 360.0 * 2 ** zoom)

    # Calculate y tile
    lat_rad = math.radians(lat)
    y_tile = int((1.0 - (math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi)) / 2.0 * 2 ** zoom)

    return x_tile, y_tile

In [ ]:
latitude, longitude = 24.5738010,72.9224973

x_tile, y_tile = lat_lon_to_tile(latitude, longitude, zoom_level)
print(f"Latitude: {latitude}, Longitude: {longitude}, Zoom: {zoom_level} -> X Tile: {x_tile}, Y Tile: {y_tile}")



x_tile = x_tile
y_tile = y_tile

In [ ]:
# Download the specific tile
download_map_tile(base_url, output_folder, zoom_level, x_tile, y_tile)

# CutMax Augmentation

In [ ]:
pwd

'/content'

In [ ]:
import os
import random
import cv2
import numpy as np

# Paths
negative_folder = r"C:\Users\Aatif\Downloads\Plantation_Detection_2.0\Negative Samples"
annotated_images_folder = r"C:\Users\Aatif\Downloads\Plantation_Detection_2.0\Annotated_data\train\images"
annotated_labels_folder = r"C:\Users\Aatif\Downloads\Plantation_Detection_2.0\Annotated_data\train\labels"
output_images_folder = r"C:\Users\Aatif\Downloads\Plantation_Detection_2.0\cutmix_folder\images"
output_labels_folder = r"C:\Users\Aatif\Downloads\Plantation_Detection_2.0\cutmix_folder\labels"

os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_labels_folder, exist_ok=True)

negative_images = [f for f in os.listdir(negative_folder) if f.endswith('.png') or f.endswith('.jpg')]
annotated_images = [f for f in os.listdir(annotated_images_folder) if f.endswith('.jpg') or f.endswith('.png')]

def parse_first_polygon(label_line):
    parts = label_line.strip().split()
    cls_id = int(parts[0])
    coords = list(map(float, parts[1:]))

    polygon = []
    for i in range(0, len(coords) - 1, 2):
        x, y = coords[i], coords[i + 1]
        if x == 1.0 and y == 1.0:
            break
        polygon.append((x, y))
    return cls_id, polygon

def polygon_to_mask(polygon, width, height):
    pts = np.array([[int(x * width), int(y * height)] for x, y in polygon], dtype=np.int32)
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [pts], 255)
    return mask, pts

def translate_polygon_pixel(polygon, orig_w, orig_h, dx, dy, new_w, new_h):
    translated = []
    for x, y in polygon:
        abs_x = x * orig_w + dx
        abs_y = y * orig_h + dy
        norm_x = abs_x / new_w
        norm_y = abs_y / new_h
        translated.append((norm_x, norm_y))
    return translated

def polygon_to_label_string(cls_id, polygon):
    return f"{cls_id} " + " ".join(f"{x:.6f} {y:.6f}" for x, y in polygon)

# Main loop: repeat each negative image twice
for neg_img_name in negative_images:
    for copy_index in [1, 2]:
        neg_img_path = os.path.join(negative_folder, neg_img_name)
        neg_img_orig = cv2.imread(neg_img_path)
        if neg_img_orig is None:
            continue

        img_h, img_w = neg_img_orig.shape[:2]
        all_labels = []
        neg_img = neg_img_orig.copy()

        random.shuffle(annotated_images)  # randomize

        for ann_img_name in annotated_images:
            ann_lbl_name = os.path.splitext(ann_img_name)[0] + '.txt'
            ann_img_path = os.path.join(annotated_images_folder, ann_img_name)
            ann_lbl_path = os.path.join(annotated_labels_folder, ann_lbl_name)

            ann_img = cv2.imread(ann_img_path)
            if ann_img is None:
                continue
            ann_h, ann_w = ann_img.shape[:2]

            try:
                with open(ann_lbl_path, 'r') as f:
                    lines = f.readlines()
            except:
                continue

            random.shuffle(lines)  # shuffle polygons

            for line in lines:
                try:
                    cls_id, polygon = parse_first_polygon(line)
                    if len(polygon) < 3:
                        continue

                    mask, polygon_pts = polygon_to_mask(polygon, ann_w, ann_h)
                    object_region = cv2.bitwise_and(ann_img, ann_img, mask=mask)

                    x, y, w, h = cv2.boundingRect(polygon_pts)
                    x_end = min(x + w, ann_w)
                    y_end = min(y + h, ann_h)
                    x = max(x, 0)
                    y = max(y, 0)
                    w = x_end - x
                    h = y_end - y

                    if w <= 1 or h <= 1 or w >= img_w or h >= img_h:
                        continue

                    crop = object_region[y:y+h, x:x+w]
                    mask_crop = mask[y:y+h, x:x+w]

                    paste_x = random.randint(0, img_w - w)
                    paste_y = random.randint(0, img_h - h)
                    roi = neg_img[paste_y:paste_y+h, paste_x:paste_x+w]

                    if len(mask_crop.shape) != 2:
                        mask_crop = cv2.cvtColor(mask_crop, cv2.COLOR_BGR2GRAY)
                    mask_crop = mask_crop.astype(np.uint8)
                    if mask_crop.shape[:2] != roi.shape[:2]:
                        continue

                    inverse_mask = cv2.bitwise_not(mask_crop)
                    bg = cv2.bitwise_and(roi, roi, mask=inverse_mask)
                    fg = cv2.bitwise_and(crop, crop, mask=mask_crop)
                    combined = cv2.add(bg, fg)
                    neg_img[paste_y:paste_y+h, paste_x:paste_x+w] = combined

                    dx = paste_x - x
                    dy = paste_y - y
                    translated_polygon = translate_polygon_pixel(polygon, ann_w, ann_h, dx, dy, img_w, img_h)
                    label_str = polygon_to_label_string(cls_id, translated_polygon)
                    all_labels.append(label_str)
                    break  # ✅ one polygon per image

                except Exception as e:
                    print(f"⚠️ Skipping polygon in {ann_lbl_name}: {e}")
                    continue

            if all_labels:
                break  # ✅ one annotation per image

        if all_labels:
            out_img_name = os.path.splitext(neg_img_name)[0] + f'_{copy_index}.jpg'
            out_img_path = os.path.join(output_images_folder, out_img_name)
            cv2.imwrite(out_img_path, neg_img)

            out_lbl_name = os.path.splitext(out_img_name)[0] + '.txt'
            out_lbl_path = os.path.join(output_labels_folder, out_lbl_name)
            with open(out_lbl_path, 'w') as f:
                for lbl in all_labels:
                    f.write(lbl + '\n')
        else:
            print(f"⚠️ Skipped: {neg_img_name}_{copy_index} (no valid object found)")

print("✅ All 200 CutMix images with single polygon annotations generated.")

# Visualization

In [ ]:
pwd

In [ ]:
import matplotlib.pyplot as plt

annotation_file_path = "C:/Users/Aatif/Downloads/Plantation_Detection_2.0/cutmix_folder/labels/tile_17_92126_56238_1.txt"
#annotation_file_path = os.path.join(pwd, annotation_file_path)
image_file_path = "C:/Users/Aatif/Downloads/Plantation_Detection_2.0/cutmix_folder/images/tile_17_92126_56238_1.jpg"
#image_file_path = os.path.join(pwd, image_file_path)

# Check if the files exist
if not os.path.exists(image_file_path):
    raise FileNotFoundError(f"Image file not found: {image_file_path}")
if not os.path.exists(annotation_file_path):
    raise FileNotFoundError(f"Annotation file not found: {annotation_file_path}")

# Read the image
image = cv2.imread(image_file_path)
if image is None:
    raise ValueError(f"Failed to read image from file: {image_file_path}")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Get the image dimensions
image_height, image_width = image.shape[:2]

# Read the annotation file
with open(annotation_file_path, 'r') as f:
    annotations = f.readlines()

# Define a function to draw polygons on the image
def draw_polygons(image, annotations, img_width, img_height):
    for annotation in annotations:
        parts = annotation.strip().split()
        class_index = int(parts[0])
        points = list(map(float, parts[1:]))
        # Scale the normalized coordinates to the image dimensions
        scaled_points = [int(points[i] * img_width if i % 2 == 0 else points[i] * img_height) for i in range(len(points))]
        # Create a list of (x, y) tuples
        polygon = [(scaled_points[i], scaled_points[i+1]) for i in range(0, len(scaled_points), 2)]
        polygon = np.array(polygon, np.int32)
        polygon = polygon.reshape((-1, 1, 2))
        # Draw the polygon on the image
        cv2.polylines(image, [polygon], isClosed=True, color=(0, 255, 0), thickness=2)
    return image

# Draw the annotations on the image
image_with_annotations = draw_polygons(image, annotations, image_width, image_height)

# Display the image with annotations
plt.figure(figsize=(10, 10))
plt.imshow(image_with_annotations)
plt.axis('off')
plt.show()

# Model Training

In [ ]:
pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

In [ ]:
# Load a model
model = YOLO("yolo11x-seg.yaml")  # build a new model from YAML
model = YOLO("yolo11x-seg.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11x-seg.yaml").load("yolo11x.pt")  # build from YAML and transfer weights

Transferred 1015/1077 items from pretrained weights


In [ ]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#Define a project --> Destination directory for all results
project = "plantations_results"
#Define subdirectory for this specific training
name = "100epochs_noAlbumentations_NEW3.0" #note that if you run the training again, it creates a directory: 200_epochs-2

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Train the model
results = model.train(data="data.yaml",
                      project=project,
                      name=name,
                      epochs=100,
                      patience=0, #I am setting patience=0 to disable early stopping.
                      batch=2,
                      imgsz=256,
                      augment=False
                      )

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-seg.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=100epochs_noAlbumentations_NEW3.0, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=0, perspective=0.0, plots=T

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              


  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  8                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  9                  -1  1   1476864  ultralytics.nn.modules.block.SPPF            [768, 768, 5]                 
 10                  -1  2   3264768  ultralytics.nn.modules.block.C2PSA           [768, 768, 2]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  2   5612544  ultralytics.nn.modules.block.C3k2            [1536, 768, 2, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

train: Scanning /content/drive/MyDrive/Pond/Plantations2.0/data/train/labels... 1027 images, 15 backgrounds, 0 corrupt: 100%|██████████| 1027/1027 [00:42<00:00, 23.93it/s]


...

    100/100      18.9G     0.5304     0.8605     0.5011      1.005          1        256: 100%|██████████| 514/514 [01:25<00:00,  5.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 11.96it/s]

                   all         72        122      0.804      0.746      0.853      0.676      0.825      0.762       0.87      0.624



100 epochs completed in 2.477 hours.
Optimizer stripped from plantations_results/100epochs_noAlbumentations_NEW3.0/weights/last.pt, 124.7MB
Optimizer stripped from plantations_results/100epochs_noAlbumentations_NEW3.0/weights/best.pt, 124.7MB

Validating plantations_results/100epochs_noAlbumentations_NEW3.0/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11x-seg summary (fused): 203 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:01<00:00, 17.24it/s]


                   all         72        122      0.788      0.803      0.852      0.679      0.812      0.828      0.882      0.624
Speed: 0.1ms preprocess, 8.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to plantations_results/100epochs_noAlbumentations_NEW3.0


# Download for Inference

In [ ]:
pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

In [ ]:
topLeft = [72.69808639301122,24.480774461878802]
topRight = [73.09634078754247,24.480774461878802]
bottomLeft = [73.09634078754247,24.932396812155133]
bottomRight = [72.69808639301122,24.932396812155133]

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def getCoords(zoomLevel):
    topleft = deg2num(topLeft[1], topLeft[0], zoomLevel)  # (c,b)
    topright = deg2num(topRight[1], topRight[0], zoomLevel)  # (a,b)
    bottomright = deg2num(bottomRight[1], bottomRight[0], zoomLevel)  # (a,d)
    bottomleft = deg2num(bottomLeft[1], bottomLeft[0], zoomLevel)  # (c,d)
    xmin = min(topleft[0], topright[0], bottomleft[0], bottomright[0])
    xmax = max(topleft[0], topright[0], bottomleft[0], bottomright[0])
    ymin = min(topleft[1], topright[1], bottomleft[1], bottomright[1])
    ymax = max(topleft[1], topright[1], bottomleft[1], bottomright[1])
    return (xmin, xmax, ymin, ymax)

def download_map_tiles(base_url, output_folder, zoom_level, scale):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get start time
    start_time = time.time()

    # Iterate over each tile within the specified range
    xmin, xmax, ymin, ymax = getCoords(zoom_level)
    for x in range(xmin, xmax + 1):
        for y in range(ymin, ymax + 1):
            # Construct the URL for the current tile with scale=3 for 640x640
            tile_url = f"{base_url}&x={x}&y={y}&z={zoom_level}&scale={scale}"
            print(tile_url)
            try:
                # Send HTTP GET request to fetch the tile
                response = requests.get(tile_url)

                # Check if request was successful (status code 200)
                if response.status_code == 200:
                    # Save the tile image to a file in the output folder
                    filename = f"tile_{zoom_level}_{x}_{y}.png"
                    filepath = os.path.join(output_folder, filename)
                    with open(filepath, "wb") as f:
                        f.write(response.content)
                    print(f"Downloaded: {filename}")
                else:
                    print(f"Failed to download tile ({x}, {y}), HTTP status code: {response.status_code}")

            except Exception as e:
                print(f"Error downloading tile ({x}, {y}): {e}")

    # Get end time
    end_time = time.time()

    # Print the total execution time
    print(f"Total time taken: {end_time - start_time} seconds")

In [ ]:
base_url = "https://mt1.google.com/vt/lyrs=s"
zoom_level = 17       # Zoom level
scale = 1           # scale of 1 = 256*256 dimensional image

output_folder = "/content/drive/MyDrive/Pond/Plantations2.0/Test/Pindwara2"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created the folder: {output_folder}")
else:
    print(f"The folder already exists: {output_folder}")


Created the folder: /content/drive/MyDrive/Pond/Plantations2.0/Test/Pindwara2


In [ ]:
download_map_tiles(base_url, output_folder, zoom_level, scale)

Streaming output truncated to the last 5000 lines.
Downloaded: tile_17_92136_56205.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56206&z=17&scale=1
Downloaded: tile_17_92136_56206.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56207&z=17&scale=1
Downloaded: tile_17_92136_56207.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56208&z=17&scale=1
Downloaded: tile_17_92136_56208.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56209&z=17&scale=1
Downloaded: tile_17_92136_56209.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56210&z=17&scale=1
Downloaded: tile_17_92136_56210.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56211&z=17&scale=1
Downloaded: tile_17_92136_56211.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56212&z=17&scale=1
Downloaded: tile_17_92136_56212.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56213&z=17&scale=1
Downloaded: tile_17_92136_56213.png
https://mt1.google.com/vt/lyrs=s&x=92136&y=56214&z=17&scale=1
Downloaded: tile_17_92136_56214.png
https://mt1.google.com/vt/lyrs=

# Inference

In [ ]:
pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

In [ ]:
model_path = "plantations_results/100epochs_noAlbumentations_NEW3.0/weights/best.pt"
model_path = os.path.join(pwd, model_path)

image_dir = "Test/Paithan"
#image_dir = os.path.join(pwd, image_dir)

annotated_images_dir = "Test_Images/Paithan2"
# Create annotated images directory if it doesn't exist
os.makedirs(annotated_images_dir, exist_ok=True)

csv_file = "Paithan2.csv"

zoom  = 17

In [ ]:
# Define class-specific confidence thresholds
conf_thresholds = {
    #'plantations': 0.61,
    'plantations': 0.5,

}

# Class names (ensure these match the order used in your model training)
class_names = [
    'plantations',

]

# Mapping of class names to abbreviations
class_abbreviations = {
    'plantations': 'P',

}

Load the model

In [ ]:
my_new_model = YOLO(model_path)

FUNCTIONS

In [ ]:
def process_image(image_path, conf_thresholds):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return None, None, None, None, None

    results = my_new_model.predict(img)

    polygons = []
    pred_classes = []
    conf_scores = []

    if results[0].masks is not None:
        for i, (polygon, cls, conf) in enumerate(zip(results[0].masks.xy, results[0].boxes.cls.cpu().numpy(), results[0].boxes.conf.cpu().numpy())):
            class_name = class_names[int(cls)]
            if conf >= conf_thresholds[class_name]:
                polygons.append(polygon)
                pred_classes.append(class_name)
                conf_scores.append(conf)

    return image_path, len(polygons), polygons, pred_classes, conf_scores

def extract_xtile_ytile(image_path):
    try:
        basename = os.path.basename(image_path)
        parts = basename.split('_')
        if len(parts) >= 4:
            xtile = int(parts[2])
            ytile = int(parts[3].split('.')[0].split()[0])
            return xtile, ytile
        else:
            raise ValueError("Filename does not contain valid tile coordinates")
    except Exception as e:
        raise ValueError(f"Filename {image_path} does not contain valid tile coordinates: {e}")

def tile_corners_to_latlon(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad_nw = math.atan(math.sinh(math.pi * (1 - 2 * (ytile / n))))
    lat_deg_nw = math.degrees(lat_rad_nw)

    lat_rad_se = math.atan(math.sinh(math.pi * (1 - 2 * ((ytile + 1) / n))))
    lat_deg_se = math.degrees(lat_rad_se)

    lat_deg_nw = max(min(lat_deg_nw, 85.0511), -85.0511)
    lat_deg_se = max(min(lat_deg_se, 85.0511), -85.0511)

    top_left = (lat_deg_nw, lon_deg)
    top_right = (lat_deg_nw, lon_deg + (360.0 / n))
    bottom_right = (lat_deg_se, lon_deg + (360.0 / n))
    bottom_left = (lat_deg_se, lon_deg)

    return top_left, top_right, bottom_left, bottom_right

def calculate_tile_center(top_left, top_right, bottom_left, bottom_right):
    center_lat = (top_left[0] + bottom_left[0]) / 2
    center_lon = (top_left[1] + top_right[1]) / 2
    return (center_lat, center_lon)


Predictions

In [ ]:
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

In [ ]:
len(image_files)

1292

In [ ]:
# Start timing
start_time = time.time()

max_vertices = 0
for image_path in image_files:
    _, _, polygons, _, _ = process_image(image_path, conf_thresholds)
    for polygon in polygons:
        max_vertices = max(max_vertices, len(polygon))

with open(csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    header = ["Image Path", "Predicted Class", "Center Latitude", "Center Longitude",
              "Top Left Latitude", "Top Left Longitude", "Top Right Latitude", "Top Right Longitude",
              "Bottom Left Latitude", "Bottom Left Longitude", "Bottom Right Latitude", "Bottom Right Longitude"]
    for i in range(1, max_vertices + 1):
        header.extend([f'X_{i}', f'Y_{i}'])
    csvwriter.writerow(header)

    for image_path in image_files:
        try:
            image_path, num_polygons, polygons, pred_classes, _ = process_image(image_path, conf_thresholds)
            if image_path is None:
                continue

            xtile, ytile = extract_xtile_ytile(image_path)
            top_left, top_right, bottom_left, bottom_right = tile_corners_to_latlon(xtile, ytile, zoom)
            latitude, longitude = calculate_tile_center(top_left, top_right, bottom_left, bottom_right)

            for pred_class, polygon in zip(pred_classes, polygons):
                row = [image_path, pred_class, latitude, longitude, top_left[0], top_left[1], top_right[0], top_right[1],
                       bottom_left[0], bottom_left[1], bottom_right[0], bottom_right[1]]
                for point in polygon:
                    row.extend([point[0], point[1]])
                csvwriter.writerow(row)
        except ValueError as e:
            print(e)
            continue

# End timing
end_time = time.time()

# Print the time taken
time_taken = end_time - start_time
print(f"CSV file '{csv_file}' saved successfully.")
print(f"Time taken to complete: {time_taken:.2f} seconds.")

Streaming output truncated to the last 5000 lines.

0: 256x256 3 Plantationss, 20.0ms
Speed: 0.7ms preprocess, 20.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 2 Plantationss, 20.7ms
Speed: 0.8ms preprocess, 20.7ms inference, 2.2ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 2 Plantationss, 20.6ms
Speed: 0.7ms preprocess, 20.6ms inference, 2.3ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 32.6ms
Speed: 0.8ms preprocess, 32.6ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 23.5ms
Speed: 0.8ms preprocess, 23.5ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 21.4ms
Speed: 0.7ms preprocess, 21.4ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Plantations, 19.6ms
Speed: 1.2ms preprocess, 19.6ms inference, 2.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detecti

## For saving annotated predictions with confidence score

In [ ]:
# pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

In [ ]:
# def visualize_polygons(image_path, polygons, pred_classes, conf_scores):
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f"Error: Unable to load image {image_path}")
#         return

#     if len(polygons) == 0:
#         print(f"No predictions for image {image_path}, not saving.")
#         return

#     for i, polygon in enumerate(polygons):
#         polygon = np.array(polygon, dtype=np.int32)
#         polygon = polygon.reshape((-1, 1, 2))
#         cv2.polylines(img, [polygon], isClosed=True, color=(0, 255, 0), thickness=2)

#         # Calculate the centroid of the polygon for placing the text
#         M = cv2.moments(polygon)
#         if M["m00"] != 0:
#             cX = int(M["m10"] / M["m00"])
#             cY = int(M["m01"] / M["m00"])
#         else:
#             cX, cY = polygon[0][0]

#         # Put class abbreviation and confidence score text on the image
#         class_abbr = class_abbreviations[pred_classes[i]]
#         conf_text = f"{class_abbr}: {conf_scores[i]:.2f}"
#         cv2.putText(img, conf_text, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

#     output_path = os.path.join(annotated_images_dir, os.path.basename(image_path))
#     cv2.imwrite(output_path, img)
#     print(f"Annotated image saved to {output_path}")

In [ ]:
# for image_path in image_files:
#     image_path, num_polygons, polygons, pred_classes, conf_scores = process_image(image_path, conf_thresholds)
#     if image_path is None:
#         continue

#     visualize_polygons(image_path, polygons, pred_classes, conf_scores)

# Convert to Geometry


In [ ]:
pwd

'/content/drive/MyDrive/Pond/Plantations2.0'

In [ ]:
# Load the CSV file
df = pd.read_csv("Paithan2.csv")

df.rename(columns={'Predicted Class': 'Class'}, inplace=True)


In [ ]:
# Constants
ZOOM_LEVEL = 17
EARTH_CIRCUMFERENCE_DEGREES = 360  # degrees

In [ ]:

def degrees_per_pixel(zoom):
    total_pixels = 256 * (2 ** zoom)
    degrees_per_pixel = EARTH_CIRCUMFERENCE_DEGREES / total_pixels
    return degrees_per_pixel

def pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, img_width, img_height):
    lon_range = lon_bottom_right - lon_top_left
    lat_range = lat_top_left - lat_bottom_right  # Note: latitude decreases as you go south
    lon = lon_top_left + (x / img_width) * lon_range
    lat = lat_top_left - (y / img_height) * lat_range  # y increases downward in image coordinates
    return lon, lat

# Initialize an empty list to store GeoJSON features
geojson_features = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    # Extract bounding coordinates of the image tile
    lat_top_left = row['Top Left Latitude']
    lon_top_left = row['Top Left Longitude']
    lat_bottom_right = row['Bottom Right Latitude']
    lon_bottom_right = row['Bottom Right Longitude']

    tile_width, tile_height = 256, 256

    # Initialize an empty list to store coordinates of the current object
    object_coords = []

    for i in range(1, 8759):  # Adjust this range according to your data
        x_col = f'X_{i}'
        y_col = f'Y_{i}'
        if x_col in row and y_col in row:
            x = row[x_col]
            y = row[y_col]
            if pd.notna(x) and pd.notna(y):
                # Convert pixel coordinates to geographic coordinates
                lon, lat = pixel_to_geo(x, y, lat_top_left, lon_top_left, lat_bottom_right, lon_bottom_right, tile_width, tile_height)
                if np.isfinite(lon) and np.isfinite(lat):
                    object_coords.append((lon, lat))
            if pd.isna(x) or pd.isna(y) or i == 8758:  # Check if it's the end of the coordinates
                if object_coords:  # Close the current shape if it has points
                    object_coords.append(object_coords[0])  # Close the polygon
                    polygon_geometry = Polygon(object_coords)
                    feature = {
                        "type": "Feature",
                        "geometry": polygon_geometry,
                        "properties": {
                            "Class": row['Class']
                        }
                    }
                    geojson_features.append(feature)
                object_coords = []

    # After the loop, check if there are any remaining coordinates to close the last polygon
    if object_coords:
        object_coords.append(object_coords[0])  # Close the polygon
        polygon_geometry = Polygon(object_coords)
        feature = {
            "type": "Feature",
            "geometry": polygon_geometry,
            "properties": {
                "Class": row['Class']
            }
        }
        geojson_features.append(feature)

# Create the GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": geojson_features
}

# Extract geometries and properties for GeoDataFrame
geometries = [feature['geometry'] for feature in geojson['features']]
properties = [feature['properties'] for feature in geojson['features']]

# Create a GeoDataFrame using the geometries and properties
gdf_final = gpd.GeoDataFrame(properties, geometry=geometries, crs="EPSG:4326")

# Transform the GeoDataFrame to EPSG:3857 (Web Mercator)
gdf_final = gdf_final.to_crs(epsg=3857)

# Save the GeoDataFrame as a shapefile
gdf_final.to_file('Shapefiles/Paithan2.shp')

